In [1]:
%env WANDB_NOTEBOOK_NAME Train.ipynb
%env CUDA_DEVICE_ORDER PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES 1

env: WANDB_NOTEBOOK_NAME=Train.ipynb
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
from mp_20_utils import load_all_data
from tokenization import MASK_SITE
device = 'cuda'
datasets_pd, torch_datasets, site_to_ids, element_to_ids, spacegroup_to_ids, max_len = load_all_data(
    device=device, allow_retokenize=True)

In [3]:
from wyckoff_transformer import WyckoffTransformerModel, WyckoffTrainer
d_hid = 200  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 4  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 4  # number of heads in ``nn.MultiheadAttention``
dropout = 0.2  # dropout probability
n_space_groups = len(spacegroup_to_ids)
# Not all 230 space groups are present in the data
model = WyckoffTransformerModel(
    n_space_groups = n_space_groups,
    n_sites = len(site_to_ids),
    n_elements = len(element_to_ids),
    d_space_groups = 16,
    d_sites = 64,
    d_species = 64,
    nhead=nhead,
    d_hid=d_hid,
    nlayers=nlayers,
    dropout=dropout).to(device)

In [4]:
import torch
trainer = WyckoffTrainer(model, torch_datasets, max_len, torch.tensor(site_to_ids[MASK_SITE]).to(device))
trainer.train(epochs=4000, val_period=10)

wandb: Currently logged in as: kazeev. Use `wandb login --relogin` to force relogin


New best val_loss_epoch 3.80627179145813, saving the model
Epoch 10 val_loss_epoch 3.80627179145813 saved to checkpoints/2024-04-09_15-59-59/best_model_params.pt
New best val_loss_epoch 2.7428152561187744, saving the model
Epoch 20 val_loss_epoch 2.7428152561187744 saved to checkpoints/2024-04-09_15-59-59/best_model_params.pt
New best val_loss_epoch 2.3932714462280273, saving the model
Epoch 40 val_loss_epoch 2.3932714462280273 saved to checkpoints/2024-04-09_15-59-59/best_model_params.pt
New best val_loss_epoch 1.8791073560714722, saving the model
Epoch 50 val_loss_epoch 1.8791073560714722 saved to checkpoints/2024-04-09_15-59-59/best_model_params.pt
New best val_loss_epoch 1.795858383178711, saving the model
Epoch 60 val_loss_epoch 1.795858383178711 saved to checkpoints/2024-04-09_15-59-59/best_model_params.pt
New best val_loss_epoch 1.705923080444336, saving the model
Epoch 90 val_loss_epoch 1.705923080444336 saved to checkpoints/2024-04-09_15-59-59/best_model_params.pt
New best val

Traceback (most recent call last):
  File "/home/kna/WyckoffTransformer/wyckoff_transformer.py", line 180, in train
    self.train_step()
  File "/home/kna/WyckoffTransformer/wyckoff_transformer.py", line 152, in train_step
    wandb.log({"train_loss_batch": loss,
  File "/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.10/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 420, in wrapper
    return func(self, *args, **kwargs)
  File "/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.10/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 371, in wrapper_fn
    return func(self, *args, **kwargs)
  File "/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.10/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 361, in wrapper
    return func(self, *args, **kwargs)
  File "/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.10/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 1838, i